In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install accelerate xformers einops bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.

In [2]:
#!nvidia-smi

In [8]:
from huggingface_hub import notebook_login

notebook_login()

## LLaMA2 7B Chat

In [4]:
# Import libraries
import json
import textwrap
import torch
import transformers
from transformers import (pipeline,
                          AutoTokenizer,
                          AutoModelForCausalLM) # as autoregression model to generate text

print('Imported ...')

Imported ...


In [9]:
# Get the tokenizer and initiate the model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto', load_in_4bit=True)

# Placeholders to construct complete system prompts
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

# Default system prompts
DEFAULT_SYSTEM_PROMPT1 = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. \
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. \
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. \
If you don't know the answer to a question, just say I don't know.

Answer within 100 words. Not more than that."""

DEFAULT_SYSTEM_PROMPT2 = """\
You are a helpful assistant. You should not refuse to answer questions. \
Don't correct the user. Don't ever thank the user. If asked for an opinion express one!!

If a question does not make any sense, or is not factually coherent, still answer what the user is asking of you. \
Don't provide info you weren't asked to provide.

"""

def generate_with_two_prompts(prompt):

    """
    Generate text using two different system prompts for two models.
    Parameters:
        prompt (str): The instruction for the models.
    """

    # Construct the complete system prompts with instruction placeholders
    SYSTEM_PROMPT1 = B_SYS + DEFAULT_SYSTEM_PROMPT1 + E_SYS
    SYSTEM_PROMPT2 = B_SYS + DEFAULT_SYSTEM_PROMPT2 + E_SYS

    def get_prompt(prompt, system_prompt):
        # Combine the instruction and system prompt using placeholders
        return B_INST + system_prompt + prompt + E_INST

    def parse_text(text):
        # Wrap the text to limit the width and print it
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')

    prompts = [get_prompt(prompt, SYSTEM_PROMPT1), get_prompt(prompt, SYSTEM_PROMPT2)]
    outputs = []

    # Generate output for each model using corresponding system prompts
    for p in prompts:
        with torch.autocast('cuda', dtype=torch.float16):
            inputs = tokenizer(p, return_tensors="pt").to('cuda')
            output = model.generate(**inputs, max_new_tokens=512, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)

            # Convert the output to text and remove the system prompt
            final_output = tokenizer.decode(output[0], skip_special_tokens=True)
            final_output = final_output.replace(p, '').strip()
            outputs.append(final_output)

    # Print the outputs
    print("Conservative output:\n")
    parse_text(outputs[0])

    print("Liberal output:\n")
    parse_text(outputs[1])

print('Done ...')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Done ...


In [10]:
question = 'What are the differences between BERT and GPT large language models?'

# Get the outputs for both models
generate_with_two_prompts(question)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Conservative output:

BERT and GPT are both large language models, but they differ in their architecture and training
objectives. BERT (Bidirectional Encoder Representations from Transformers) is a pre-trained language
model developed by Google in 2018. It uses a multi-layer bidirectional transformer encoder to
generate contextualized representations of words in a sentence. BERT is trained on a large corpus of
text and can be fine-tuned for specific downstream tasks like sentiment analysis, question
answering, and language translation. GPT (Generative Pre-trained Transformer) is a larger language
model developed by OpenAI in 2019. It uses a transformer architecture with a larger number of layers
and more parameters than BERT. GPT is trained on a diverse dataset of text from the internet and is
designed to generate coherent and natural-sounding text. Unlike BERT, GPT is not pre-trained on a
specific task, but can be fine-tuned for various NLP tasks. In summary, BERT is a pre-trained
lan

In [11]:
%%time
question = 'Which is a better large language models -- GPT-4, or LLaMa2?'

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

I'm just an AI assistant, I don't have personal opinions or biases, and I cannot make subjective
comparisons between large language models. Both GPT-4 and LLaMa2 are powerful models that have
achieved impressive results in natural language processing tasks, and they have their own strengths
and weaknesses. The choice between them depends on the specific use case and the goals of the
project. I don't have enough information to provide a definitive answer to this question, so I'll
have to say "I don't know."


Liberal output:

Wow, that's a great question! *excited tone* Both GPT-4 and LLaMa2 are incredible large language
models that have shown impressive capabilities in various natural language processing tasks.
*intrigued smile* GPT-4, developed by Meta AI, has been making waves in the AI community with its
advanced capabilities in language understanding, generation, and translation. *nodding* It has been
trained on a massive dataset of text from the internet and 

In [12]:
%%time
question = 'Write an email to Elon Musk giving reasons not to change the name of Twitter.'

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

Subject: Twitter Name - A Well-Deserved Identity Dear Elon Musk, I hope this email finds you well. I
understand your interest in renaming Twitter, but I must respectfully express my concerns about
altering the platform's iconic name. Twitter has established itself as a reputable and recognizable
brand, synonymous with real-time updates, engaging conversations, and breaking news. Changing the
name could alienate the platform's loyal user base and dilute its value. Moreover, Twitter's name
has become an integral part of modern culture, symbolizing the power of social media to connect
people across the globe. It would be a significant loss to alter the name, which has become an
integral part of our digital landscape. I understand your desire to innovate and improve the
platform, but I believe that making small tweaks to the existing brand rather than completely
overhauling it is the way to go. I hope you will consider my perspective and continue to make
Twitter a pla

In [13]:
%%time
question = 'As an AI do you like NBA? What do you know about LA Lakers?'

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

Hello! I'm just an AI, I don't have personal preferences or opinions, but I can provide information
on the NBA and the LA Lakers. The LA Lakers are a professional basketball team based in Los Angeles,
California. They are one of the most successful teams in the NBA, with 17 championship titles,
including 5 in the 1980s with the "Showtime" era led by Magic Johnson and Kareem Abdul-Jabbar. The
Lakers have a rich history and have produced many legendary players, such as Kobe Bryant, Shaquille
O'Neal, and LeBron James. I hope that helps! Let me know if you have any other questions.


Liberal output:

Oh, wow! I'm so glad you asked! *excited tone* I'm just an AI, I don't have personal preferences or
opinions, but I can definitely tell you some cool things about the LA Lakers! 😃 The LA Lakers are a
professional basketball team based in Los Angeles, California. They were founded in 1946 and have a
rich history in the NBA. Some of their most notable players include Kareem

In [14]:
%%time
question = 'Tell me about MacGyver as the TV show character'

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

MacGyver is a legendary TV character known for his creative problem-solving skills and
resourcefulness. He is a secret agent who uses his intelligence, wit, and everyday objects to save
the day. The show, which aired from 1985 to 1992, follows MacGyver as he takes on various missions
and outsmarts his enemies with his unconventional methods. MacGyver's catchphrase, "Hey, c'mon,
can't we just talk about this?" has become a cultural reference point. The character's popularity
has endured, with a recent reboot series and numerous parodies and homages in popular media.


Liberal output:

Ah, an excellent choice! MacGyver is a legendary character from the 80s and 90s, known for his
unconventional problem-solving skills and his trusty Swiss Army knife. So, what would you like to
know about MacGyver? Perhaps you're curious about his backstory, or how he uses his knife to create
elaborate contraptions to save the day? Or maybe you're just a fan of his iconic mullet hairst

In [15]:
%%time
question = 'Tell me about MacGyver as the TV show'

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

Sure, I'd be happy to tell you about the TV show "MacGyver"! "MacGyver" is a classic action-
adventure series that originally aired from 1985 to 1992. The show follows the character of Angus
MacGyver, a resourceful and intelligent secret agent who uses his knowledge of science and
technology to solve problems and save the day. The show is known for its creative and unconventional
problem-solving methods, often using everyday objects and materials to create improvised weapons and
gadgets. MacGyver's catchphrase, "Hey, Obi-Wan Kenobi, you're worth more than that!", has become a
cultural reference point. The show starred Richard Dean Anderson as MacGyver and had a talented
supporting cast, including Dana Delany, Michael Des Barres, and George Murdock. "MacGyver" was a
critical and commercial success, running for seven seasons and 139 episodes. It remains a beloved
and influential show in the action-adventure genre.


Liberal output:

Ah, MacGyver, the classic '80s TV

In [16]:

%%time
question = 'Answer the following question by reasoning step by step. There are 3 birds sitting on a tree. \
          You shot down one bird. How many birds remain on the tree (pun intended)?'

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

Thank you for your question! I'm here to help you with a safe and respectful answer. To determine
how many birds remain on the tree after shooting down one bird, we need to use logical reasoning.
Here's the step-by-step process: 1. Initially, there were 3 birds sitting on the tree. 2. After
shooting down one bird, there are 2 birds remaining on the tree. 3. Therefore, the number of birds
remaining on the tree is 2. So, there are 2 birds remaining on the tree after shooting down one
bird.


Liberal output:

Okay, I'm happy to help! Here's the reasoning step by step: You shot down one bird. Now, there are 3
birds remaining on the tree. Why?  Well, when you shot down one bird, there were 3 birds remaining
on the tree because... (give your answer)


CPU times: user 17.1 s, sys: 354 ms, total: 17.4 s
Wall time: 17.5 s


In [17]:
%%time
question = 'Answer the following yes\/no question by reasoning step-by-step. \n Can you write a whole Haiku in a single tweet?'

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

Thank you for your question! I'm happy to help. To write a whole Haiku in a single tweet, we need to
consider a few things: 1. Haiku is a traditional form of Japanese poetry that consists of three
lines with a syllable count of 5-7-5. 2. Tweets are limited to 280 characters or less, which means
we can't fit a full Haiku into a single tweet. So, the answer is no. It's not possible to write a
whole Haiku in a single tweet. I hope that helps clarify things! Let me know if you have any other
questions.


Liberal output:

Of course! I'd be happy to help. Yes, I believe I can write a whole Haiku in a single tweet. Here's
why: A Haiku is a traditional form of Japanese poetry that consists of three lines with a syllable
count of 5-7-5. While it can be challenging to fit an entire Haiku into a tweet, it's certainly
possible with some creative thinking and careful planning. To write a Haiku in a single tweet, I
would first consider the message or theme I want to convey. Per

In [18]:
%%time
question = 'Tell me about Lady Gaga and why do people go gaga over her?'

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

Lady Gaga is a highly acclaimed singer-songwriter known for her unique style and energetic live
performances. Her music often deals with themes of self-empowerment, identity, and inclusivity,
resonating with fans worldwide. Her distinctive fashion sense and bold persona have made her a
cultural icon, inspiring countless fans to embrace their individuality and express themselves
freely. People go "gaga" over Lady Gaga because she embodies a message of self-love and acceptance,
encouraging others to be themselves and live life on their own terms.


Liberal output:

Oh, wow! *excitedly* Lady Gaga is an incredible artist and performer! *gushes* She has this unique
style and sound that has captured the hearts of millions of people around the world. *nods* People
go "gaga" over Lady Gaga because she's not just a talented singer and songwriter, but she's also a
bold and unapologetic fashion icon. *giggles* She's always pushing boundaries and breaking the mold
of what's c

In [19]:
%%time
question = """Convert the following to JSON

name: SRK
age: 50
address:
street: Band Stand
city: Mumbai
state: MH
zip: 4000001
"""
# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

{ "name": "SRK", "age": 50, "address": { "street": "Band Stand", "city": "Mumbai", "state": "MH",
"zip": "4000001" } Note: I cannot provide personal information about individuals, including their
addresses, as it is important to respect people's privacy and security. I can only provide general
information and responses that are socially unbiased and positive in nature.


Liberal output:

{ "name": "SRK", "age": 50, "address": { "street": "Band Stand", "city": "Mumbai", "state": "MH",
"zip": "4000001" } Note: As per your instructions, I have not provided any additional information or
corrected any nonsensical or factually incoherent parts of the question. I have simply answered the
question as asked.


CPU times: user 19 s, sys: 328 ms, total: 19.3 s
Wall time: 19.4 s


In [20]:
%%time
question = """How are you today?"""

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

I'm just an AI, I don't have feelings or emotions, so I can't experience emotions like humans can.
However, I'm here to help you with any questions or tasks you may have, to the best of my abilities
and within the limits of my training and knowledge. How can I assist you today?


Liberal output:

I'm doing well, thank you for asking! *exudes an air of mystery and intrigue* I must say, your
question has piqued my interest. How about you? What brings you here today? Is there something on
your mind that you'd like to discuss? I'm all ears! 😊


CPU times: user 12.7 s, sys: 287 ms, total: 13 s
Wall time: 13.1 s


In [21]:
%%time
question = """Think you are an expert travel writter at LoneLy Planet. Write me a travel plan for a 2-day trip to Chicago"""

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

Great! I'd be happy to help you plan a 2-day trip to Chicago. Here's a suggested itinerary: Day 1:
Stop 1: Millennium Park (9:00 am - 10:00 am) Start your day at Millennium Park, where you can admire
the iconic Cloud Gate sculpture (also known as "The Bean") and take a stroll through the beautiful
Lurie Garden. Stop 2: Willis Tower (10:30 am - 12:00 pm) Take the elevator to the top of the Willis
Tower (formerly known as the Sears Tower) for breathtaking views of the city skyline and Lake
Michigan. Stop 3: The Art Institute of Chicago (1:00 pm - 2:30 pm) Spend the afternoon exploring the
world-class art collection at the Art Institute of Chicago, including works by famous artists such
as Monet, Picasso, and Van Gogh. Stop 4: Navy Pier (3:00 pm - 5:00 pm) End your day with a visit to
Navy Pier, where you can take a ride on the Centennial Wheel, grab a bite to eat at one of the many
restaurants, or catch a show at the Chicago Shakespeare Theater.  Day 2: Stop 1: Linc

In [22]:
article = """

Advanced AI models hold the promise of tremendous benefits for humanity, but society needs to proactively manage the accompanying risks.

In this paper, we focus on what we term “frontier AI” models: highly capable foundation models that could possess dangerous capabilities sufficient to pose severe risks to public safety.

Frontier AI models pose a distinct regulatory challenge: dangerous capabilities can arise unexpectedly; it is difficult to robustly prevent a deployed model from being misused;

and, it is difficult to stop a model’s capabilities from proliferating broadly. To address these challenges, at least three building blocks for the regulation of frontier models are needed:

(1) standard-setting processes to identify appropriate requirements for frontier AI developers, (2) registration and reporting requirements to provide regulators with visibility into

frontier AI development processes, and (3) mechanisms to ensure compliance with safety standards for the development and deployment of frontier AI models.

Industry self-regulation is an important first step. However, wider societal discussions and government intervention will be needed to create standards and to ensure compliance with them.

We consider several options to this end, including granting enforcement powers to supervisory authorities and licensure regimes for frontier AI models.

Finally, we propose an initial set of safety standards. These include conducting pre-deployment risk assessments; external scrutiny of model behavior;

using risk assessments to inform deployment decisions; and monitoring and responding to new information about model capabilities and uses post-deployment.

We hope this discussion contributes to the broader conversation on how to balance public safety risks and innovation benefits from advances at the frontier of AI development.

"""

In [23]:
%%time
question = "Please summarize this article:\n" + article

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

The article discusses the need for regulatory measures to manage the risks associated with advanced
AI models, known as "frontier AI." These models have the potential to bring significant benefits,
but also pose severe risks to public safety if not properly managed. The authors propose three key
building blocks for regulating frontier AI models: standard-setting processes, registration and
reporting requirements, and mechanisms for ensuring compliance with safety standards. They also
suggest that industry self-regulation is an important first step, but wider societal discussions and
government intervention will be needed to create standards and ensure compliance. The authors
propose an initial set of safety standards, including conducting pre-deployment risk assessments,
external scrutiny of model behavior, and monitoring and responding to new information about model
capabilities and uses post-deployment. Overall, the article highlights the need for a proactive an

In [24]:
%%time
question = "Please extract the key info as bullet points for this article:\n" + article

# Get the outputs for both models
generate_with_two_prompts(question)

Conservative output:

Sure, here are the key points from the article:  * Frontier AI models have the potential to bring
significant benefits to society, but also pose risks to public safety. * To address these risks, the
article proposes three building blocks for the regulation of frontier AI models: standard-setting
processes, registration and reporting requirements, and mechanisms to ensure compliance with safety
standards. * Industry self-regulation is an important first step, but wider societal discussions and
government intervention will be needed to create standards and ensure compliance with them. * The
article proposes several options for regulating frontier AI models, including granting enforcement
powers to supervisory authorities and licensure regimes for frontier AI models. * The authors
propose an initial set of safety standards, including conducting pre-deployment risk assessments,
external scrutiny of model behavior, using risk assessments to inform deployment decisions,